# TELL Runs for the National Transmission Planning (NTP) Project

This notebook executes the initial set of runs of the TELL model for the NTP project.

In [ ]:
# Start by importing the TELL package and information about your operating system:
import os 
import tell
import sklearn

import pandas as pd
import matplotlib.pyplot as plt

from tell.package_data import get_ba_abbreviations
from glob import glob


## Set the Directory Structure

In [ ]:
# Identify the top-level directory and the subdirectory where the data will be stored:
current_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs'
tell_data_dir = os.path.join(current_dir, r'tell_data')

# If the "tell_data_dir" subdirectory doesn't exist then create it:
if not os.path.exists(tell_data_dir):
   os.makedirs(tell_data_dir)


## Run the MLP Models

In [ ]:
# Generate a list of BA abbreviations to process:
ba_abbrev_list = tell.get_balancing_authority_to_model_dict().keys()

scenario_to_process = 'historic'

# Run the MLP prediction step for the list of BAs using parallel processing streams:
for year_to_process in range(1980,2020,1):
    pdf = tell.predict_batch(target_region_list = ba_abbrev_list,
                             year = year_to_process,
                             data_dir = os.path.join(tell_data_dir, r'wrf_to_tell_data', scenario_to_process),
                             datetime_field_name = 'Time_UTC',
                             save_prediction = True,
                             prediction_output_directory = os.path.join(tell_data_dir, r'outputs', r'mlp_output', scenario_to_process),
                             n_jobs = -1)


## Run the Forward Execution

In [ ]:
# Run the TELL model forward in time for a given year in five year increments out to 2100:
for year in range(1980,2020,1):
    summary_df, ba_time_series_df, state_time_series_df = tell.execute_forward(year_to_process = str(year),
                                                                               gcam_target_year = str(year), 
                                                                               scenario_to_process = 'historic',
                                                                               data_output_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/outputs/tell_output',
                                                                               gcam_usa_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/gcamusa_data',
                                                                               map_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/ba_service_territory_data',
                                                                               mlp_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/outputs/mlp_output',
                                                                               pop_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/population_data',
                                                                               save_county_data = False)


## Clean Up the Results

In [ ]:
# Define function to aggregate the MLP output into a single file:
def aggregate_mlp_output(data_input_dir: str):
    
    # Create a list of all county meteorology files in the input directory:
    list_of_files = glob(f'{data_input_dir}/*_mlp_output.csv')
    
    # Loop over that list and extract the population-weighted T2 values:
    for file in range(len(list_of_files)):
        # Read in the .csv file:
        load_df = pd.read_csv(list_of_files[file])
        
        # Rename the load variable for clarity:
        load_df.rename(columns={'Load': 'Load_MWh'}, inplace=True)
        
        # Aggregate the output into a new dataframe:
        if file == 0:
            output_df = load_df
        else:
            output_df = pd.concat([output_df, load_df])
    
    # Reorder and sort the columns:
    output_df = output_df[['BA', 'Time_UTC', 'Load_MWh']]
    output_df = output_df.sort_values(['BA', 'Time_UTC'])
    
    # Generate the .csv output file name:
    csv_output_filename = os.path.join(data_input_dir, 'All_BA_Output.csv')

    # Write out the dataframe to a .csv file:
    output_df.to_csv(csv_output_filename, sep=',', index=False)


In [ ]:
# Identify the directory to aggregate:
data_input_dir =  '/Users/burl878/Documents/Code/code_repos/ntp_heat_wave/data/tell_data/without_population_effects/rcp85hotter_ssp5'

# Aggregate the individual MLP output files into a single .csv file:
aggregate_mlp_output(data_input_dir = data_input_dir)


In [ ]:
# Set the top-level data input and output directory:
data_input_dir =  '/Users/burl878/Documents/Code/code_repos/ntp_heat_wave/data/tell_data/without_population_effects'

# Read in the input files, rename the load variable, and aggregate the data across scenarios:
historic_df = pd.read_csv((data_input_dir + '/historic/' + 'All_BA_Output.csv'))
historic_df.rename(columns={'Load_MWh': 'historic'}, inplace=True)

# Generate the .csv output file name:
csv_output_filename = os.path.join(data_input_dir, 'BA_Loads_2006_Without_Population_Effects.csv')

# Write out the dataframe to a .csv file:
historic_df.to_csv(csv_output_filename, sep=',', index=False)

historic_df


In [ ]:
# Set the top-level data input and output directory:
data_input_dir =  '/Users/burl878/Documents/Code/code_repos/ntp_heat_wave/data/tell_data/without_population_effects'

# Read in the input files, rename the load variable, and aggregate the data across scenarios:
rcp45cooler_ssp3 = pd.read_csv((data_input_dir + '/rcp45cooler_ssp3/' + 'All_BA_Output.csv'))
rcp45cooler_ssp3.rename(columns={'Load_MWh': 'rcp45cooler_ssp3'}, inplace=True)
output_df = rcp45cooler_ssp3.copy()

rcp45cooler_ssp5 = pd.read_csv((data_input_dir + '/rcp45cooler_ssp5/' + 'All_BA_Output.csv'))
rcp45cooler_ssp5.rename(columns={'Load_MWh': 'rcp45cooler_ssp5'}, inplace=True)
output_df = output_df.merge(rcp45cooler_ssp5, on=['BA', 'Time_UTC'])

rcp45hotter_ssp3 = pd.read_csv((data_input_dir + '/rcp45hotter_ssp3/' + 'All_BA_Output.csv'))
rcp45hotter_ssp3.rename(columns={'Load_MWh': 'rcp45hotter_ssp3'}, inplace=True)
output_df = output_df.merge(rcp45hotter_ssp3, on=['BA', 'Time_UTC'])

rcp45hotter_ssp5 = pd.read_csv((data_input_dir + '/rcp45hotter_ssp5/' + 'All_BA_Output.csv'))
rcp45hotter_ssp5.rename(columns={'Load_MWh': 'rcp45hotter_ssp5'}, inplace=True)
output_df = output_df.merge(rcp45hotter_ssp5, on=['BA', 'Time_UTC'])

rcp85cooler_ssp3 = pd.read_csv((data_input_dir + '/rcp85cooler_ssp3/' + 'All_BA_Output.csv'))
rcp85cooler_ssp3.rename(columns={'Load_MWh': 'rcp85cooler_ssp3'}, inplace=True)
output_df = output_df.merge(rcp85cooler_ssp3, on=['BA', 'Time_UTC'])

rcp85cooler_ssp5 = pd.read_csv((data_input_dir + '/rcp85cooler_ssp5/' + 'All_BA_Output.csv'))
rcp85cooler_ssp5.rename(columns={'Load_MWh': 'rcp85cooler_ssp5'}, inplace=True)
output_df = output_df.merge(rcp85cooler_ssp5, on=['BA', 'Time_UTC'])

rcp85hotter_ssp3 = pd.read_csv((data_input_dir + '/rcp85hotter_ssp3/' + 'All_BA_Output.csv'))
rcp85hotter_ssp3.rename(columns={'Load_MWh': 'rcp85hotter_ssp3'}, inplace=True)
output_df = output_df.merge(rcp85hotter_ssp3, on=['BA', 'Time_UTC'])

rcp85hotter_ssp5 = pd.read_csv((data_input_dir + '/rcp85hotter_ssp5/' + 'All_BA_Output.csv'))
rcp85hotter_ssp5.rename(columns={'Load_MWh': 'rcp85hotter_ssp5'}, inplace=True)
output_df = output_df.merge(rcp85hotter_ssp5, on=['BA', 'Time_UTC'])

# Generate the .csv output file name:
csv_output_filename = os.path.join(data_input_dir, 'BA_Loads_2046_Without_Population_Effects.csv')

# Write out the dataframe to a .csv file:
output_df.to_csv(csv_output_filename, sep=',', index=False)

output_df


In [ ]:
# Set the top-level data input and output directory:
data_input_dir =  '/Users/burl878/Documents/Code/code_repos/ntp_heat_wave/data/tell_data/with_population_effects'

# Read in the input files, rename the load variable, and aggregate the data across scenarios:
historic_df = pd.read_csv((data_input_dir + '/historic/' + 'TELL_Balancing_Authority_Hourly_Load_Data_2006_Scaled_2006.csv'))
historic_df.rename(columns={'Scaled_TELL_BA_Load_MWh': 'historic','BA_Code': 'BA'}, inplace=True)
historic_df = historic_df[['BA', 'Time_UTC', 'historic']]

# Generate the .csv output file name:
csv_output_filename = os.path.join(data_input_dir, 'BA_Loads_2006_With_Population_Effects.csv')

# Write out the dataframe to a .csv file:
historic_df.to_csv(csv_output_filename, sep=',', index=False)

historic_df


In [ ]:
# Set the top-level data input and output directory:
data_input_dir =  '/Users/burl878/Documents/Code/code_repos/ntp_heat_wave/data/tell_data/with_population_effects'

# Read in the input files, rename the load variable, and aggregate the data across scenarios:
rcp45cooler_ssp3 = pd.read_csv((data_input_dir + '/rcp45cooler_ssp3/' + 'TELL_Balancing_Authority_Hourly_Load_Data_2046_Scaled_2046.csv'))
rcp45cooler_ssp3.rename(columns={'Scaled_TELL_BA_Load_MWh': 'rcp45cooler_ssp3','BA_Code': 'BA'}, inplace=True)
rcp45cooler_ssp3 = rcp45cooler_ssp3[['BA', 'Time_UTC', 'rcp45cooler_ssp3']]
output_df = rcp45cooler_ssp3.copy()

rcp45cooler_ssp5 = pd.read_csv((data_input_dir + '/rcp45cooler_ssp5/' + 'TELL_Balancing_Authority_Hourly_Load_Data_2046_Scaled_2046.csv'))
rcp45cooler_ssp5.rename(columns={'Scaled_TELL_BA_Load_MWh': 'rcp45cooler_ssp5','BA_Code': 'BA'}, inplace=True)
rcp45cooler_ssp5 = rcp45cooler_ssp5[['BA', 'Time_UTC', 'rcp45cooler_ssp5']]
output_df = output_df.merge(rcp45cooler_ssp5, on=['BA', 'Time_UTC'])

rcp45hotter_ssp3 = pd.read_csv((data_input_dir + '/rcp45hotter_ssp3/' + 'TELL_Balancing_Authority_Hourly_Load_Data_2046_Scaled_2046.csv'))
rcp45hotter_ssp3.rename(columns={'Scaled_TELL_BA_Load_MWh': 'rcp45hotter_ssp3','BA_Code': 'BA'}, inplace=True)
rcp45hotter_ssp3 = rcp45hotter_ssp3[['BA', 'Time_UTC', 'rcp45hotter_ssp3']]
output_df = output_df.merge(rcp45hotter_ssp3, on=['BA', 'Time_UTC'])

rcp45hotter_ssp5 = pd.read_csv((data_input_dir + '/rcp45hotter_ssp5/' + 'TELL_Balancing_Authority_Hourly_Load_Data_2046_Scaled_2046.csv'))
rcp45hotter_ssp5.rename(columns={'Scaled_TELL_BA_Load_MWh': 'rcp45hotter_ssp5','BA_Code': 'BA'}, inplace=True)
rcp45hotter_ssp5 = rcp45hotter_ssp5[['BA', 'Time_UTC', 'rcp45hotter_ssp5']]
output_df = output_df.merge(rcp45hotter_ssp5, on=['BA', 'Time_UTC'])

rcp85cooler_ssp3 = pd.read_csv((data_input_dir + '/rcp85cooler_ssp3/' + 'TELL_Balancing_Authority_Hourly_Load_Data_2046_Scaled_2046.csv'))
rcp85cooler_ssp3.rename(columns={'Scaled_TELL_BA_Load_MWh': 'rcp85cooler_ssp3','BA_Code': 'BA'}, inplace=True)
rcp85cooler_ssp3 = rcp85cooler_ssp3[['BA', 'Time_UTC', 'rcp85cooler_ssp3']]
output_df = output_df.merge(rcp85cooler_ssp3, on=['BA', 'Time_UTC'])

rcp85cooler_ssp5 = pd.read_csv((data_input_dir + '/rcp85cooler_ssp5/' + 'TELL_Balancing_Authority_Hourly_Load_Data_2046_Scaled_2046.csv'))
rcp85cooler_ssp5.rename(columns={'Scaled_TELL_BA_Load_MWh': 'rcp85cooler_ssp5','BA_Code': 'BA'}, inplace=True)
rcp85cooler_ssp5 = rcp85cooler_ssp5[['BA', 'Time_UTC', 'rcp85cooler_ssp5']]
output_df = output_df.merge(rcp85cooler_ssp5, on=['BA', 'Time_UTC'])

rcp85hotter_ssp3 = pd.read_csv((data_input_dir + '/rcp85hotter_ssp3/' + 'TELL_Balancing_Authority_Hourly_Load_Data_2046_Scaled_2046.csv'))
rcp85hotter_ssp3.rename(columns={'Scaled_TELL_BA_Load_MWh': 'rcp85hotter_ssp3','BA_Code': 'BA'}, inplace=True)
rcp85hotter_ssp3 = rcp85hotter_ssp3[['BA', 'Time_UTC', 'rcp85hotter_ssp3']]
output_df = output_df.merge(rcp85hotter_ssp3, on=['BA', 'Time_UTC'])

rcp85hotter_ssp5 = pd.read_csv((data_input_dir + '/rcp85hotter_ssp5/' + 'TELL_Balancing_Authority_Hourly_Load_Data_2046_Scaled_2046.csv'))
rcp85hotter_ssp5.rename(columns={'Scaled_TELL_BA_Load_MWh': 'rcp85hotter_ssp5','BA_Code': 'BA'}, inplace=True)
rcp85hotter_ssp5 = rcp85hotter_ssp5[['BA', 'Time_UTC', 'rcp85hotter_ssp5']]
output_df = output_df.merge(rcp85hotter_ssp5, on=['BA', 'Time_UTC'])

# Generate the .csv output file name:
csv_output_filename = os.path.join(data_input_dir, 'BA_Loads_2046_With_Population_Effects.csv')

# Write out the dataframe to a .csv file:
output_df.to_csv(csv_output_filename, sep=',', index=False)

output_df
